In [ ]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
from DFTStructureGenerator import DFThandle
import os
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
import pickle
np.random.seed(0)

In [ ]:
binol_3_3 = [1,2,3,13,18,19,20,21,23,24,25,26,28,29,30] + np.arange(122, 131).tolist()
binol_4_4 = [62,63,66]
binol_6_6 = [31,32,35,36,] + np.arange(44, 48).tolist()
binol_7_7 = [51,52,56,57,60]
binol_other = [0,85,91,121]
all_binol_split = [binol_3_3, binol_4_4, binol_6_6, binol_7_7, binol_other]

In [ ]:
allowed_ligand = [1,3,4,5,6,9,10,11,12,14,15,16,17,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,44,45,46,48,49,50,51,52,54,56,57,58,59,61,62,63,64,65,66,67,68,69,70,
71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,91,92,93,94,95,96,97,98,99,100,
101,102,104,105,106,107,108,109,110,111,113,114,117,118,122,125,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,
151,152,153,154,155,156,157,158,159,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205]
banned_binol = [188, 195, 220, 676]

In [ ]:
data_dir = "Data"
csv_dir = os.path.join(data_dir, "Iteration_2", "Result")
row_csv = "Data_clear.csv"
target_csv = "Data_clear_with_sites.csv"

In [ ]:
with open(r"Data/all_fp_map2.pkl", 'rb')as f:
    # rd_mf_map, rd_des_map, morgan_map, modred_map, acsf_3D_map, soap_3D_map, mbtr_3D_map, lmbtr_3D_map, qm_dict, area_dict = pickle.load(f)
    qm_dict, area_dict = pickle.load(f)

# 1. Full Reaction Space Conclusion

In [ ]:
smiles_csv = pd.read_csv(target_csv)
binol_csv = smiles_csv.loc[smiles_csv['Type'] == 'Binol']
ligand_csv = smiles_csv.loc[(smiles_csv['Type'] == 'Ligand_Box') | (smiles_csv['Type'] == 'Ligand_Other')]
ligand_csv = ligand_csv.loc[np.isin(ligand_csv['Index'] - 1000, allowed_ligand)]

In [ ]:
smiles_csv = pd.read_csv("Data_clear_with_sites.csv")
binol_csv = smiles_csv.loc[smiles_csv['Type'] == 'Binol']
ligand_csv = smiles_csv.loc[(smiles_csv['Type'] == 'Ligand_Box') | (smiles_csv['Type'] == 'Ligand_Other')]
new_dict = {"Binol":{}, "Ligand":{}, "Binol_smiles":{},"Ligand_smiles":{}, "Binol_conf_id":{}, "Ligand_conf_id":{}, "Binol_Sites":{}, "Ligand_Sites":{}, "Binol_G":{}, "Ligand_G":{}}
new_dict_id = 0
for _, binol_row in binol_csv.iterrows():
    for _, ligand_row in ligand_csv.iterrows():
        if ligand_row['Index'] - 1000 not in allowed_ligand:
            continue
        new_dict["Binol"][new_dict_id] = binol_row['Index']
        new_dict["Ligand"][new_dict_id] = ligand_row['Index']
        new_dict["Binol_smiles"][new_dict_id] = binol_row['Smiles']
        new_dict["Ligand_smiles"][new_dict_id] = ligand_row['Smiles']
        new_dict["Binol_conf_id"][new_dict_id] = binol_row['conf_id']
        new_dict["Ligand_conf_id"][new_dict_id] = ligand_row['conf_id']
        new_dict["Binol_Sites"][new_dict_id] = binol_row['Sites']
        new_dict["Ligand_Sites"][new_dict_id] = ligand_row['Sites']
        new_dict["Binol_G"][new_dict_id] = binol_row['G/Hatree']
        new_dict["Ligand_G"][new_dict_id] = ligand_row['G/Hatree']
        new_dict_id += 1
new_dict = pd.DataFrame(new_dict)
new_dict.to_csv("Data/Iteration_2/Full_Space_20250925.csv", index=False)

# 2. Active Learning

In [ ]:
full_data_csv = DFThandle.read_reaction_csv("Data/Full_Space_new.csv")
data_clear_csv = pd.read_csv("Data_clear_with_sites.csv")
fps = [AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(row['Smiles']), 2, nBits=2048) for row_id, row in data_clear_csv.iterrows()]
mfps = np.array(fps) @ np.array(fps).T
row_id_idx = {row["Index"]: row_id for row_id, row in data_clear_csv.iterrows()}
target_idx = [[row_id_idx[row["Binol"]], row_id_idx[row["Ligand"]]] for row_id, row in full_data_csv.iterrows()]

## Init Round

In [ ]:
from rdkit.SimDivFilters.rdSimDivPickers import MaxMinPicker
from rdkit import DataStructs
def distij(i,j):
    return 1 - DataStructs.TanimotoSimilarity(fps[i], fps[j])
    
def distanceij(i,j):
    B_i, L_i = target_idx[i]
    B_j, L_j = target_idx[j]
    return distij(B_i, B_j) + distij(L_i, L_j)#  * rate_a # + var_j * rate_b
Times = 0      
picker = MaxMinPicker()
first_picker_idx = picker.LazyPick(distanceij, len(full_data_csv), 50, firstPicks=[])
np.save(f'Data/Iteration_Data2/iter_{Times:05}.npy', first_picker_idx)
full_data_csv.iloc[first_picker_idx].to_csv(f"Data/Iteration_Data2/iter_{Times:05}_sameBinol.csv", index=False)

## Next Round(1-5)

In [ ]:
from rdkit.SimDivFilters.rdSimDivPickers import MaxMinPicker
from rdkit import DataStructs

def distij(i,j):
    return 1 - DataStructs.TanimotoSimilarity(fps[i], fps[j])

def generate_similarity(B_i, L_i, B_j, L_j):
    k = mfps[B_i, B_j] + mfps[L_i, L_j]
    return k / (mfps[B_i, B_i] + mfps[L_i, L_i] + mfps[B_j, B_j] + mfps[L_j, L_j] - k)

def distanceij(i,j):
    B_i, L_i = target_idx[i]
    B_j, L_j = target_idx[j]
    var_i = this_uncertainty[i]
    return distij(B_i, B_j) + distij(L_i, L_j) + var_i + avaliable(B_i, L_i) * 1000

def get_uncertainty(first_data_csv):
    train_X = DFThandle.descriptor_to_array(first_data_csv, None, [qm_dict, area_dict])
    train_Y = first_data_csv['R'].to_numpy() - first_data_csv["S"].to_numpy()
    temp_train_X =  DFThandle.get_reverse_result(train_X)
    temp_train_Y = train_Y * -1
    train_X = np.concatenate([train_X, temp_train_X])
    train_Y = np.append(train_Y, temp_train_Y)
    
    model = CatBoostRegressor(task_type="CPU", iterations=10000, learning_rate=0.01, depth=6, verbose=0, random_state = 0, loss_function='RMSEWithUncertainty')
    model.fit(train_X, train_Y)
    new_desc_array = DFThandle.descriptor_to_array(full_data_csv, None, [qm_dict, area_dict])
    new_result, new_uncertainty = model.predict(new_desc_array).T
    return new_result, new_uncertainty

In [ ]:
Times = 5
first_data_csv = DFThandle.read_reaction_csv(f"Data/Iteration_2/Result/BINOL_result_sum_{Times-1:04}.csv")
first_idxs = np.load(f"Data/Iteration_2/Iteration_Data/iter_{Times-1:05}.npy")
all_result, this_uncertainty = get_uncertainty(first_data_csv)
full_data_csv['Pred_Value'] = all_result
pre_select_id = first_idxs.tolist()
picker = MaxMinPicker()
picker_idx = picker.LazyPick(distanceij, len(full_data_csv), len(first_idxs) + 50, firstPicks=pre_select_id)
np.save(f'Data/Iteration_2/Iteration_Data/iter_{Times:05}.npy', picker_idx)
full_data_csv.iloc[picker_idx[-50:]].to_csv(f"Data/Iteration_2/Iteration_Data/iter_{Times:05}.csv", index=False)

## Next Round (6 - )

In [ ]:
data_csv = first_data_csv
y = data_csv['R'].to_numpy() - data_csv["S"].to_numpy()
all_X = DFThandle.descriptor_to_array(data_csv, None, [qm_dict, area_dict])
target = y
all_pred = np.zeros(len(all_X))
kf = KFold(n_splits=5, shuffle=True, random_state=1)
kf = list(kf.split(all_X))
r2_split = []
for train_ids, test_ids in kf:
    model = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=6, verbose=0, random_state = 0)
    train_X, train_Y = all_X[train_ids], target[train_ids]
    temp_train_X =  DFThandle.get_reverse_result(train_X)
    temp_train_Y = train_Y * -1
    train_X_ = np.concatenate([train_X, temp_train_X])
    train_Y_ = np.append(train_Y, temp_train_Y)
    model.fit(train_X_, train_Y_)
    y_pred = model.predict(all_X[test_ids])
    all_pred[test_ids] = y_pred
for binol_split_id, binol_split in enumerate(all_binol_split):
    row_in_split = [id_ for id_, each in data_csv.iterrows() if each['Binol'] in binol_split]
    r2_split.append(r2_score(target[row_in_split], all_pred[row_in_split]))

In [ ]:
import numpy as np

def stratified_sample_allocation(n, weights):
    weights = np.array(weights)
    expected = n * weights
    floors = np.floor(expected).astype(int)
    fractions = expected - floors
    remaining = n - np.sum(floors)

    sorted_indices = np.argsort(-fractions)
    
    allocations = floors.copy()
    allocations[sorted_indices[:remaining]] += 1
    return allocations

weight = (1 - np.array(r2_split)) * np.array([len(each) for each in all_binol_split])
weight /= np.sum(weight)
binol_sample_num = stratified_sample_allocation(50, weight)
binol_sample_num

In [ ]:
def avaliable(B_j, L_j):
    if B_j in binol:
        return 1.0
    else:
        return 0.0

In [ ]:
Times = 15
first_idxs = np.load(f"Data/Iteration_2/Iteration_Data/iter_{Times-1:05}.npy")
picker_idx = first_idxs.tolist()
for binol, binol_sample in zip(all_binol_split, binol_sample_num.tolist()):
    if binol_sample <= 0:
        continue
    picker = MaxMinPicker()
    picker_idx = picker.LazyPick(distanceij, len(full_data_csv), len(picker_idx) + binol_sample, firstPicks=list(picker_idx))
full_data_csv['Pred_Value'] = all_result
np.save(f'Data/Iteration_2/Iteration_Data/iter_{Times:05}.npy', picker_idx)
full_data_csv.iloc[list(picker_idx)[-np.sum(binol_sample_num):]].to_csv(f"Data/Iteration_2/Iteration_Data/iter_{Times:05}.csv", index=False)

# Final Recommendation

In [ ]:

smiles_csv = pd.read_csv("Data_clear_with_sites.csv")
binol_csv = smiles_csv.loc[smiles_csv['Type'] == 'Binol']
important_binol_id = np.arange(max(binol_csv['Index'].to_list())+1)
# important_binol_id = [802,803,804]
full_data_csv = pd.read_csv('Data/Iteration_2/Full_Space_20250925.csv')
for id_ in range(14,15):
    first_data_csv = DFThandle.read_reaction_csv(rf'Data\Iteration_2\Result\BINOL_result_sum_{id_:04}.csv')
    old_desc_array = DFThandle.descriptor_to_array(first_data_csv, None, [qm_dict, area_dict])
    old_y = first_data_csv['R'].to_numpy() - first_data_csv["S"].to_numpy()
    model = CatBoostRegressor(task_type="CPU", iterations=10000, learning_rate=0.01, depth=6, verbose=0, random_state = 0)
    model.fit(old_desc_array, old_y)
    new_desc_array = DFThandle.descriptor_to_array(full_data_csv, None, [qm_dict, area_dict])
    new_y = model.predict(new_desc_array).T
    full_data_csv['PredER'] = new_y
    target_ids = []
    for binol_id in important_binol_id:
    # for binol_id in [24]:
        temp_csv = full_data_csv.loc[full_data_csv['Binol'] == binol_id]
        already_ligand = first_data_csv.loc[first_data_csv['Binol'] == binol_id]['Ligand'].to_list()
        # temp_csv = full_data_csv.loc[full_data_csv['In Dataset'] == 0]
        temp_csv['PredERAbs'] = np.abs(temp_csv['PredER'])
        temp_csv = temp_csv.sort_values(by='PredERAbs', ascending=False)
        temp_csv = temp_csv.loc[~np.isin(temp_csv["Ligand"], already_ligand)]
        target_ids +=[row_id for row_id, row in temp_csv.iterrows()][:10]
        # temp_csv = temp_csv.sort_values(by='PredER', ascending=True)
        # target_ids += [row_id for row_id, row in temp_csv.iterrows()][:3]
    # target_ids = list(set(target_ids))
    new_csv = full_data_csv.iloc[target_ids]
    new_csv['PredE'] = (new_csv['PredER'] + 100) / 2
    new_csv['PredR'] = (new_csv['PredER'] - 100) / 2
    new_csv.to_csv(f"Pred_Important_Binol_09_25_{id_:04}.csv", index=False)